## Custom Segmentation Algorithm Profling

Notebook to profile segmentation implementation ([spheroid_cytometer.py](spheroid_cytometer.py)) for performance deficiencies.

In [1]:
%matplotlib inline
%run spheroid_cytometer.py
import os
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from scipy import ndimage
from skimage import feature
from scipy import ndimage
from skimage import morphology
from skimage import filters
from cytokit.ops import cytometry as cytometer_op
from cytokit.ops import tile_generator
from cytokit import math as ck_math
from cytokit import config as ck_config
from itkwidgets import view

In [2]:
exp_name = '20190215-mc38-dmso-control'
exp_dir = 'XY07'
data_dir = osp.join(os.environ['CYTOKIT_DATA_DIR'], 'cytokit', 'mc38-spheroid', exp_name, exp_dir, 'raw')
config_dir = osp.join(os.environ['CYTOKIT_REPO_DIR'], 'pub', 'config', 'mc38-spheroid')
config = ck_config.load(config_dir)
os.environ['CYTOKIT_PATH_FORMATS'] = 'get_default_path_formats("1_' + exp_dir + '_{tile:05d}_Z{z:03d}_CH{channel:d}.tif")'
config.register_environment()

In [3]:
tile_gen = tile_generator.CytokitTileGenerator(config, data_dir, 0, 4)

In [4]:
tile = tile_gen.run()
tile.shape

(1, 26, 3, 1440, 1920)

In [5]:
op = cytometer_op.Cytometry2D(config)

In [6]:
op.initialize()

In [7]:
# %%time
# img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=None, min_object_size=1024, min_peak_dist=200)
# img_seg.shape

In [8]:
%%time
img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=.5, min_object_size=512, min_peak_dist=75)
img_seg.shape

CPU times: user 1min 6s, sys: 11.7 s, total: 1min 17s
Wall time: 1min 17s


In [9]:
#view(img_seg[:, 5].astype(np.float32).copy())

In [10]:
%%time
stats = op.cytometer.quantify(tile, img_seg, sigma=(.1, 1, 1), channel_names=config.channel_names, 
                              cell_graph=True, morphology_features=False, 
                              cell_intensity=['mean', 'sum'])

CPU times: user 4min 4s, sys: 3.28 s, total: 4min 7s
Wall time: 4min 7s


In [11]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 26 columns):
id                     371 non-null int64
x                      371 non-null float64
y                      371 non-null float64
z                      371 non-null int64
cm:size                371 non-null int64
cm:diameter            371 non-null float64
cm:perimeter           371 non-null float64
cm:solidity            371 non-null float64
cm:circularity         371 non-null float64
nm:size                371 non-null int64
nm:diameter            371 non-null float64
nm:perimeter           371 non-null float64
nm:solidity            371 non-null float64
nm:circularity         371 non-null float64
cg:n_neighbors         371 non-null int64
cg:neighbor_ids        371 non-null object
cg:adj_neighbor_pct    371 non-null object
cg:adj_bg_pct          371 non-null object
ci:BF:mean             371 non-null float64
ci:LIVE:mean           371 non-null float64
ci:DEAD:mean           371 n

In [12]:
stats['id'].nunique()

23

In [15]:
#!pip install line_profiler
%load_ext line_profiler

In [18]:
%lprun -f op.cytometer.quantify stats = op.cytometer.quantify(tile, img_seg, channel_names=config.channel_names, cell_graph=True, morphology_features=False, cell_intensity=['mean', 'sum'])
#%lprun -f op.cytometer.segment img_seg = op.cytometer.segment(tile[0, :, 0])

Timer unit: 1e-06 s

Total time: 265.727 s
File: /lab/repos/cytokit/pub/analysis/mc38-spheroid/spheroid_cytometer.py
Function: quantify at line 131

Line #      Hits         Time  Per Hit   % Time  Line Contents
   131